In [4]:
import pandas as pd
from typing import TYPE_CHECKING, Any, Dict, List, Mapping, Sequence, cast


In [2]:
data = pd.read_parquet('/home/isma/repos/book/data-pipelines-with-airflow-2nd-ed/chapter13_genai/recipe_book/notebooks/splitted_with_vectors.parquet')
data

,filename,document_sha,chunk_sha,chunk,vector
0,empanadas-argentinas.txt,ec598303-9510-5d6e-9c5e-ccb43fcf8c4e,c001f008-b5df-5ec0-8905-17ae645171e6,Empanadas argentinas,"[-0.006259406451135874, -0.005043736658990383,..."
1,empanadas-argentinas.txt,ec598303-9510-5d6e-9c5e-ccb43fcf8c4e,8830dfd7-1331-5221-8a08-5f8cc7c46c49,Ingredientes Medidas 4 Porciones Seleccionar t...,"[-0.0013423336204141378, 0.008130612783133984,..."
2,empanadas-argentinas.txt,ec598303-9510-5d6e-9c5e-ccb43fcf8c4e,9c95627f-1e0c-5a5d-8bdd-eb4e9cbb7da0,"1 taza de cebolla, finamente picada, para el r...","[0.0017024519620463252, -0.0022656326182186604..."
3,empanadas-argentinas.txt,ec598303-9510-5d6e-9c5e-ccb43fcf8c4e,b88038c3-6087-52c9-a2df-dffef9bff309,"1 taza de huevo duro, finamente picado, para e...","[-0.004608715884387493, -0.007404274772852659,..."
4,empanadas-argentinas.txt,ec598303-9510-5d6e-9c5e-ccb43fcf8c4e,1f433cc2-34d7-511d-addd-f8660e850f36,"suficiente de harina, para extender la masa su...","[-0.014491851441562176, -0.002449954394251108,..."
5,empanadas-argentinas.txt,ec598303-9510-5d6e-9c5e-ccb43fcf8c4e,c60774cf-e659-5237-a63e-3bd0aea3ccfb,Para el relleno de carne: calienta el aceite d...,"[-0.005385186523199081, -0.007785658817738295,..."
6,empanadas-argentinas.txt,ec598303-9510-5d6e-9c5e-ccb43fcf8c4e,b839d3a7-4daf-5460-975c-cb76ee3dc321,Para el relleno de elote: derrite la mantequil...,"[-0.0181081872433424, -0.008491011336445808, -..."
7,empanadas-argentinas.txt,ec598303-9510-5d6e-9c5e-ccb43fcf8c4e,e554b863-d17a-588f-9210-b05955c6f34b,Barniza las empanadas argentinas con el huevo ...,"[0.0062514375895261765, -0.0070929154753685, -..."
8,pavo-a-la-hawaiana.txt,3303775d-98cc-5c62-adef-be3dce52c5a4,54af8361-016d-565e-9803-dccbc11fb95d,Pavo a la hawaiana Ingredientes 1 taza de sals...,"[-0.013167427852749825, -0.0001442846114514395..."
9,pavo-a-la-hawaiana.txt,3303775d-98cc-5c62-adef-be3dce52c5a4,367213bc-c570-5935-be25-ed7993c225b8,"Preparación En un bowl, combina la salsa de so...","[-0.012540347874164581, -0.019292842596769333,..."


In [11]:
uuid_column = "chunk_sha"

unique_columns = sorted(data.columns.to_list())

data = data.drop_duplicates(subset=[uuid_column], keep="first")



In [14]:
def _prepare_document_to_uuid_map(
    data: Sequence[Mapping], group_key: str, get_value
) -> dict[str, set]:
    """Prepare the map of grouped_key to set."""
    grouped_key_to_set: dict = {}
    for item in data:
        document_url = item[group_key]

        if document_url not in grouped_key_to_set:
            grouped_key_to_set[document_url] = set()

        grouped_key_to_set[document_url].add(get_value(item))
    return grouped_key_to_set


input_documents_to_uuid = _prepare_document_to_uuid_map(
    data=data.to_dict("records"),
    group_key="chunk",
    get_value=lambda x: x["chunk_sha"],
)

input_documents_to_uuid



{'Empanadas argentinas': {'c001f008-b5df-5ec0-8905-17ae645171e6'},
 'Ingredientes Medidas 4 Porciones Seleccionar todos los ingredientes 2 tazas de harina de trigo, para la masa 1 cucharadita de sal, para la masa 1 cucharadita de polvo para hornear, para la masa 1/2 tazas de mantequilla, en cubos y a temperatura ambiente, para la masa 1/4 tazas de vinagre blanco, para la masa 1 taza de agua de anís, para la masa 2 cucharadas de aceite de oliva, para el relleno 1 taza de cebolla, finamente picada, para el relleno de carne': {'8830dfd7-1331-5221-8a08-5f8cc7c46c49'},
 '1 taza de cebolla, finamente picada, para el relleno de carne 1 cucharada de ajo, finamente picado, para el relleno de carne 1/4 tazas de pimiento rojo, en cubos pequeños, para el relleno de carne 2 tazas de carne molida de res, para el relleno de carne sal y pimienta al gusto 2 cucharaditas de orégano, para el relleno de carne 2 cucharaditas de comino, para el relleno de carne 1 taza de huevo duro, finamente picado, para e

In [ ]:
documents_to_uuid: dict = {}
document_keys = set(data[document_column])
while True:
    collection = self.get_collection(collection_name)
    data_objects = collection.query.fetch_objects(
        filters=Filter.any_of(
            [Filter.by_property(document_column).equal(key) for key in document_keys]
        ),
        return_properties=[document_column],
        limit=limit,
        offset=offset,
    )
    if len(data_objects.objects) == 0:
        break
    offset = offset + limit

    if uuid_column in data_objects.objects[0].properties:
        data_object_properties = [obj.properties for obj in data_objects.objects]
    else:
        data_object_properties = []
        for obj in data_objects.objects:
            row = dict(obj.properties)
            row[uuid_column] = str(obj.uuid)
            data_object_properties.append(row)

    documents_to_uuid.update(
        self._prepare_document_to_uuid_map(
            data=data_object_properties,
            group_key=document_column,
            get_value=lambda x: x[uuid_column],
        )
    )


In [ ]:
\
changed_documents = set()
unchanged_docs = set()
new_documents = set()


# segregate documents into changed, unchanged and non-existing documents.
for doc_url, doc_set in input_documents_to_uuid.items():
    if doc_url in existing_documents_to_uuid
        if existing_documents_to_uuid[doc_url] != doc_set:
            changed_documents.add(str(doc_url))
        else:
            unchanged_docs.add(str(doc_url))
    else:
        new_documents.add(str(doc_url))

In [ ]:

    (
        documents_to_uuid_map,
        changed_documents,
        unchanged_documents,
        new_documents,
    ) = self._get_segregated_documents(
        data=data,
        document_column=document_column,
        uuid_column=uuid_column,
        collection_name=collection_name,
    )
    if verbose:
        self.log.info(
            "Found %s changed documents, %s unchanged documents and %s non-existing documents",
            len(changed_documents),
            len(unchanged_documents),
            len(new_documents),
        )
        for document in changed_documents:
            self.log.info(
                "Changed document: %s has %s objects.", document, len(documents_to_uuid_map[document])
            )
        self.log.info("Non-existing document: %s", ", ".join(new_documents))

    if existing == "error" and len(changed_documents):
        raise ValueError(
            f"Documents {', '.join(changed_documents)} already exists. You can either skip or replace"
            f" them by passing 'existing=skip' or 'existing=replace' respectively."
        )
    elif existing == "skip":
        data = data[data[document_column].isin(new_documents)]
        if verbose:
            self.log.info(
                "Since existing=skip, ingesting only non-existing document's object %s", data.shape[0]
            )
    elif existing == "replace":
        total_objects_count = sum([len(documents_to_uuid_map[doc]) for doc in changed_documents])
        if verbose:
            self.log.info(
                "Since existing='replace', deleting %s objects belonging changed documents %s",
                total_objects_count,
                changed_documents,
            )
        if list(changed_documents):
            batch_delete_error = self._delete_all_documents_objects(
                document_keys=list(changed_documents),
                document_column=document_column,
                collection_name=collection_name,
                total_objects_count=total_objects_count,
                batch_delete_error=batch_delete_error,
                verbose=verbose,
            )
        data = data[data[document_column].isin(new_documents.union(changed_documents))]
        self.log.info("Batch inserting %s objects for non-existing and changed documents.", data.shape[0])

    if data.shape[0]:
        self.batch_data(
            collection_name=collection_name,
            data=data,
            vector_col=vector_column,
            uuid_col=uuid_column,
        )
        if batch_delete_error:
            if batch_delete_error:
                self.log.info("Failed to delete %s objects.", len(batch_delete_error))
            # Rollback object that were not created properly
            self._delete_objects(
                [item["uuid"] for item in batch_delete_error],
                collection_name=collection_name,
            )

    if verbose:
        collection = self.get_collection(collection_name)
        self.log.info(
            "Total objects in collection %s : %s ",
            collection_name,
            collection.aggregate.over_all(total_count=True),
        )